In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='Malgun Gothic')
rc('axes', unicode_minus=False)

In [2]:
import datetime
import os
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import json

In [3]:
date_today = datetime.date.today()
date_key = date_today.strftime('%y%m%d')

#### 종목 코드 save by market

In [4]:
market_code = 'stockMkt'
# 'kospi': 'stockMkt',
# 'kosdaq': 'kosdaqMkt',
# 'konex': 'konexMkt'

In [5]:
sto_code_url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=' + market_code

In [6]:
sto_code_df = pd.read_html(sto_code_url)[0]
sto_code_df[:5]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,JS전선,5560,절연선 및 케이블 제조업,"선박선,고무선,전력선,통신선 제조",2007-11-12,12월,이익희,http://www.jscable.co.kr,충청남도
1,거북선2호,101380,NaN,운송장비(선박) 임대,2008-04-25,12월,신주선,NaN,부산광역시
2,거북선6호,114140,NaN,NaN,2009-10-01,12월,김연신,NaN,제주특별자치도
3,교보메리츠,64900,NaN,"부동산 투자,운용",2002-01-30,12월,김 상 진,NaN,서울특별시
4,국제관광공사,28780,NaN,NaN,1966-03-18,12월,NaN,NaN,NaN


In [7]:
save_n_code_df = ('data/code_list/' + 'code_list_' + market_code + '_' + date_key + '.csv').strip()
save_n_code_df

'data/code_list/code_list_stockMkt_190801.csv'

In [8]:
sto_code_df.to_csv(save_n_code_df, encoding='euc-kr', index=False) #cp949
print('save >> ' + save_n_code_df)

save >> data/code_list/code_list_stockMkt_190801.csv


#### 종목코드 load

In [9]:
os.listdir('data/code_list')

['code_list_190728.csv',
 'code_list_190729.csv',
 'code_list_190730.csv',
 'code_list_stockMkt_190730.csv',
 'code_list_stockMkt_190801.csv']

In [10]:
select_file_code_df = ('data/code_list/' + os.listdir('data/code_list')[-1])
select_file_code_df

'data/code_list/code_list_stockMkt_190801.csv'

In [11]:
sto_code_df = pd.read_csv(select_file_code_df, encoding='euc-kr')
sto_code_df[:3]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,JS전선,5560,절연선 및 케이블 제조업,"선박선,고무선,전력선,통신선 제조",2007-11-12,12월,이익희,http://www.jscable.co.kr,충청남도
1,거북선2호,101380,NaN,운송장비(선박) 임대,2008-04-25,12월,신주선,NaN,부산광역시
2,거북선6호,114140,NaN,NaN,2009-10-01,12월,김연신,NaN,제주특별자치도


In [12]:
sto_code_df['종목코드'] = sto_code_df['종목코드'].map('{:06d}'.format)

In [13]:
sto_code_sort = sto_code_df[['회사명', '종목코드']]
sto_code_sort[-5:]

,회사명,종목코드
3730,지앤이헬스케어,299480
3731,툴젠,199800
3732,틸론,217880
3733,판도라티비,202960
3734,한국비엔씨,226610


In [14]:
sto_code_sort.shape

(3735, 2)

#### sise 

In [208]:
def url_creater(what) :
    target_url = []
    target_code = []
#   target = ['국내', '해외', '환율', 'WTI', '휘발유', '국제금', '국내금']
    url_base = 'https://finance.naver.com/'
    url_ref = {'국내' : 'sise/sise_index_day.nhn?code=', 
              '해외' : 'world/worldDayListJson.nhn?symbol=', 
              '환율' : 'marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_', 
              'WTI' : 'marketindex/worldDailyQuote.nhn?marketindexCd=', 
              '휘발유' : 'marketindex/oilDailyQuote.nhn?marketindexCd=', 
              '국제금' : 'marketindex/worldDailyQuote.nhn?marketindexCd=', 
              '국내금' : 'marketindex/goldDailyQuote.nhn?'}
    cate_code = {'국내' : ['KOSPI', 'KOSDAQ', 'KPI200'], 
              '해외' : ['DJI@DJI', 'NII@NI225', 'LNS@FTSE100', 
                      'NAS@IXIC', 'SHS@000001', 'PAS@CAC40', 
                      'SPI@SPX', 'HSI@HSI', 'XTR@DAX30' ], 
              '환율' : ['USDKRW', 'JPYKRW', 'EURKRW', 'CNYKRW'], 
              'WTI' : ['OIL_CL'], 
              '휘발유' : ['OIL_GSL'], 
              '국제금' : ['CMDT_GC'], 
              '국내금' : ['']}
    pager = {'국내' : '&page=', 
              '해외' : '&fdtc=0&page=', 
              '환율' : '&page=', 
              'WTI' : '&fdtc=2&page=', 
              '휘발유' : '&page=', 
              '국제금' : '&fdtc=2&page=', 
              '국내금' : '&page='}
    for i in range(len(cate_code[what])) :
        target_url += [url_base + url_ref[what] + cate_code[what][i] + pager[what]]
        target_code += [cate_code[what][i]]
    return target_url, target_code
    

In [218]:
url_creater('국내')[1][2]

'KPI200'

In [210]:
def last_page_finder(url):
    target_html = BeautifulSoup(urlopen(url).read(), 'lxml') #html.parser
    page_pkg = target_html.find_all("table") #,align="center"
    if page_pkg ==[] : return 500
    else :
        last_page_pkg = page_pkg[0].find_all("td",class_="pgRR")
        if last_page_pkg ==[] : 
            last_page_pkg = page_pkg[1].find_all("td",class_="pgRR")
        last_page_pkg = last_page_pkg[0].a.get('href')
        last_page_num = int(last_page_pkg.split("=")[-1].strip())
        return last_page_num

In [232]:
def info_collector_html(market) :
    info_df = pd.DataFrame()
    target_url = url_creater(market)
    k = -1
#     last_page_num = last_page_finder(target_url+'1')
    for i in target_url[0] :
        temp_df = pd.DataFrame()
        k += 1
        last_page_num = last_page_finder(i+'1')
        for j in range(1, last_page_num+1) :
            page_url = (f'{i}{j}')
            temp_df = temp_df.append(pd.read_html(page_url, header=0)[0], 
                                     ignore_index=True)
            temp_df = temp_df.dropna(axis=0)

        temp_df.set_index('날짜', inplace=True)
        temp_df.columns = (temp_df.columns + '_' + target_url[1][k])
        info_df = pd.concat([info_df, temp_df], axis=1, sort=False)
    return info_df    

In [224]:
k = 0
for i in range(5):
    print(i)
    k += 1
    print(k)
    print()

0
1

1
2

2
3

3
4

4
5



In [125]:
# def info_collector_html(base_url, code) :
#     info_df = pd.DataFrame()
#     target_url = base_url + code
#     last_page_num = last_page_finder(target_url)
#     for i in range(1, last_page_num+1) :
#         page_url = (f'{target_url}&page={i}')
#         info_df = info_df.append(pd.read_html(page_url, header=0)[0], ignore_index=True)
#         info_df = info_df.dropna(axis=0)
#     return info_df    

In [196]:
# 해외지수 전용 (no table)
def info_collector_ajax(base_url, code) :
    info_df = pd.DataFrame()
    target_url = base_url + code
    last_page_num = last_page_finder(target_url)
    cols = {"xymd":"날짜","clos":'종가',"diff":'전일비',
            "open":'시가',"high":'고가',"low":'저가',
            "rate":"rate","gvol":"gvol", "symb":"symb"}
    for i in range(1, last_page_num+1) :
        page_url = (f'{target_url}&fdtc=0&page={i}')
        info_df = info_df.append(pd.DataFrame(requests.get(page_url).json()), ignore_index=True)
        info_df = info_df.dropna(axis=0)
        info_df = info_df.drop_duplicates(['xymd'])
    info_df.rename(columns=cols, inplace=True)
    return info_df     

[지수]

* 국내

In [17]:
base_url_jisu_do = 'https://finance.naver.com/sise/sise_index_day.nhn?code='

In [18]:
code_jisu_do = ['KOSPI', 'KOSDAQ', 'KPI200']

In [67]:
jisu_do_info_df = pd.DataFrame()
for i in code_jisu_do :
    temp = info_collector_html(base_url_jisu_do, i)
    temp.set_index('날짜', inplace=True)
    temp.columns = (temp.columns + '_' + i)
    jisu_do_info_df = pd.concat([jisu_do_info_df, temp], axis=1, sort=False)
jisu_do_info_df

,체결가_KOSPI,전일비_KOSPI,등락률_KOSPI,거래량(천주)_KOSPI,거래대금(백만)_KOSPI,체결가_KOSDAQ,전일비_KOSDAQ,등락률_KOSDAQ,거래량(천주)_KOSDAQ,거래대금(백만)_KOSDAQ,체결가_KPI200,전일비_KPI200,등락률_KPI200,거래량(천주)_KPI200,거래대금(백만)_KPI200
2019.08.01,2017.30,7.25,-0.36%,415701.0,3870951.0,622.47,7.71,-1.22%,630719.0,3786046.0,266.04,0.30,-0.11%,64143.0,2537888.0
2019.07.31,2024.55,14.13,-0.69%,578033.0,5415666.0,630.18,4.54,+0.73%,755260.0,4160117.0,266.34,2.61,-0.97%,91703.0,3731560.0
2019.07.30,2038.68,9.20,+0.45%,538759.0,3821938.0,625.64,6.86,+1.11%,704143.0,3462591.0,268.95,1.20,+0.45%,63169.0,2334005.0
2019.07.29,2029.48,36.78,-1.78%,596166.0,4626034.0,618.78,25.81,-4.00%,771304.0,3712485.0,267.75,4.50,-1.65%,77793.0,2804706.0
2019.07.26,2066.26,8.22,-0.40%,583332.0,5056450.0,644.59,7.81,-1.20%,870552.0,3896380.0,272.25,0.79,-0.29%,88517.0,3404986.0
2019.07.25,2074.48,7.82,-0.38%,590890.0,5126430.0,652.40,7.43,-1.13%,856206.0,4105781.0,273.04,0.18,-0.07%,94460.0,3835185.0
2019.07.24,2082.30,19.15,-0.91%,841580.0,4115461.0,659.83,8.82,-1.32%,955261.0,4915984.0,273.22,2.70,-0.98%,82752.0,2919409.0
2019.07.23,2101.45,8.11,+0.39%,640162.0,4523644.0,668.65,6.13,-0.91%,884196.0,4534091.0,275.92,1.37,+0.50%,93952.0,3219043.0
2019.07.22,2093.34,1.02,-0.05%,320412.0,3156360.0,674.78,0.72,+0.11%,634148.0,3312898.0,274.55,0.00,0.00%,59881.0,2340357.0
2019.07.19,2094.36,27.81,+1.35%,386118.0,4037112.0,674.06,8.91,+1.34%,743055.0,3886505.0,274.55,3.88,+1.43%,85702.0,2956076.0


In [68]:
jisu_do_info_df.shape

(7722, 15)

In [69]:
jisu_do_info_df.isnull().sum()

체결가_KOSPI             1
전일비_KOSPI             1
등락률_KOSPI             1
거래량(천주)_KOSPI         1
거래대금(백만)_KOSPI        1
체결가_KOSDAQ         1916
전일비_KOSDAQ         1916
등락률_KOSDAQ         1916
거래량(천주)_KOSDAQ     1916
거래대금(백만)_KOSDAQ    1916
체결가_KPI200         4364
전일비_KPI200         4364
등락률_KPI200         4364
거래량(천주)_KPI200     4364
거래대금(백만)_KPI200    4364
dtype: int64

In [234]:
dome_info = info_collector_html('국내')
dome_info

,체결가_KOSPI,전일비_KOSPI,등락률_KOSPI,거래량(천주)_KOSPI,거래대금(백만)_KOSPI,체결가_KOSDAQ,전일비_KOSDAQ,등락률_KOSDAQ,거래량(천주)_KOSDAQ,거래대금(백만)_KOSDAQ,체결가_KPI200,전일비_KPI200,등락률_KPI200,거래량(천주)_KPI200,거래대금(백만)_KPI200
2019.08.01,2017.34,7.21,-0.36%,461743.0,4443149.0,622.26,7.92,-1.26%,694237.0,4161131.0,265.73,0.61,-0.23%,72578.0,2900144.0
2019.07.31,2024.55,14.13,-0.69%,578033.0,5415666.0,630.18,4.54,+0.73%,755260.0,4160117.0,266.34,2.61,-0.97%,91703.0,3731560.0
2019.07.30,2038.68,9.20,+0.45%,538759.0,3821938.0,625.64,6.86,+1.11%,704143.0,3462591.0,268.95,1.20,+0.45%,63169.0,2334005.0
2019.07.29,2029.48,36.78,-1.78%,596166.0,4626034.0,618.78,25.81,-4.00%,771304.0,3712485.0,267.75,4.50,-1.65%,77793.0,2804706.0
2019.07.26,2066.26,8.22,-0.40%,583332.0,5056450.0,644.59,7.81,-1.20%,870552.0,3896380.0,272.25,0.79,-0.29%,88517.0,3404986.0
2019.07.25,2074.48,7.82,-0.38%,590890.0,5126430.0,652.40,7.43,-1.13%,856206.0,4105781.0,273.04,0.18,-0.07%,94460.0,3835185.0
2019.07.24,2082.30,19.15,-0.91%,841580.0,4115461.0,659.83,8.82,-1.32%,955261.0,4915984.0,273.22,2.70,-0.98%,82752.0,2919409.0
2019.07.23,2101.45,8.11,+0.39%,640162.0,4523644.0,668.65,6.13,-0.91%,884196.0,4534091.0,275.92,1.37,+0.50%,93952.0,3219043.0
2019.07.22,2093.34,1.02,-0.05%,320412.0,3156360.0,674.78,0.72,+0.11%,634148.0,3312898.0,274.55,0.00,0.00%,59881.0,2340357.0
2019.07.19,2094.36,27.81,+1.35%,386118.0,4037112.0,674.06,8.91,+1.34%,743055.0,3886505.0,274.55,3.88,+1.43%,85702.0,2956076.0


In [235]:
dome_info.shape

(7722, 15)

In [236]:
dome_info.isnull().sum()

체결가_KOSPI             1
전일비_KOSPI             1
등락률_KOSPI             1
거래량(천주)_KOSPI         1
거래대금(백만)_KOSPI        1
체결가_KOSDAQ         1916
전일비_KOSDAQ         1916
등락률_KOSDAQ         1916
거래량(천주)_KOSDAQ     1916
거래대금(백만)_KOSDAQ    1916
체결가_KPI200         4364
전일비_KPI200         4364
등락률_KPI200         4364
거래량(천주)_KPI200     4364
거래대금(백만)_KPI200    4364
dtype: int64

- 해외

In [22]:
base_url_jisu_exp = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol='

In [23]:
code_jisu_exp = ['DJI@DJI', 'NII@NI225', 'LNS@FTSE100', 'NAS@IXIC', 'SHS@000001',
                'PAS@CAC40', 'SPI@SPX', 'HSI@HSI', 'XTR@DAX30' ]

In [24]:
# 다우산업 : DJI@DJI    ,    # 니케이225 : NII@NI225
# 영국 : LNS@FTSE100    ,    # 나스닥종합 : NAS@IXIC
# 상해종합 : SHS@000001 ,    # 프랑스 : PAS@CAC40
# S&P 500 : SPI@SPX     ,    # 항셍 : HSI@HSI
# 독일 : XTR@DAX30

In [38]:
jisu_exp_info_df = pd.DataFrame()
for i in code_jisu_exp :
    temp = info_collector_ajax(base_url_jisu_exp, i)
    temp.set_index('날짜', inplace=True)
    temp.columns = (temp.columns + '_' + i)
    jisu_exp_info_df = pd.concat([jisu_exp_info_df, temp], axis=1, sort=False)
jisu_exp_info_df

,종가_DJI@DJI,전일비_DJI@DJI,gvol_DJI@DJI,고가_DJI@DJI,저가_DJI@DJI,시가_DJI@DJI,rate_DJI@DJI,symb_DJI@DJI,종가_NII@NI225,전일비_NII@NI225,...,rate_HSI@HSI,symb_HSI@HSI,종가_XTR@DAX30,전일비_XTR@DAX30,gvol_XTR@DAX30,고가_XTR@DAX30,저가_XTR@DAX30,시가_XTR@DAX30,rate_XTR@DAX30,symb_XTR@DAX30
20190731,26864.27,-333.75,388998694.0,27281.65,26719.60,27244.67,-1.23,DJI@DJI,21521.53,-187.78,...,-1.31,HSI@HSI,12189.04,41.80,94019580.0,12226.49,12139.03,12149.09,0.34,XTR@DAX30
20190730,27198.02,-23.33,293245226.0,27224.36,27069.86,27145.39,-0.09,DJI@DJI,21709.31,92.51,...,0.14,HSI@HSI,12147.24,-270.23,118257935.0,12404.51,12115.28,12398.24,-2.18,XTR@DAX30
20190729,27221.35,28.90,259316069.0,27275.85,27178.06,27192.24,0.11,DJI@DJI,21616.80,-41.35,...,-1.03,HSI@HSI,12417.47,-2.43,66298553.0,12473.97,12387.58,12398.29,-0.02,XTR@DAX30
20190726,27192.45,51.47,243198930.0,27213.70,27123.25,27166.00,0.19,DJI@DJI,21658.15,-98.40,...,-0.69,HSI@HSI,12419.90,57.80,73308041.0,12428.05,12356.15,12363.86,0.47,XTR@DAX30
20190725,27140.98,-128.99,244021675.0,27298.43,27062.48,27247.39,-0.47,DJI@DJI,21756.55,46.98,...,0.25,HSI@HSI,12362.10,-160.79,119522742.0,12599.93,12299.10,12576.02,-1.28,XTR@DAX30
20190724,27269.97,-79.22,245881077.0,27291.04,27191.12,27262.24,-0.29,DJI@DJI,21709.57,88.69,...,0.20,HSI@HSI,12522.89,32.15,96088881.0,12550.34,12476.55,12510.52,0.26,XTR@DAX30
20190723,27349.19,177.29,250797407.0,27368.81,27204.58,27231.86,0.65,DJI@DJI,21620.88,204.09,...,0.34,HSI@HSI,12490.74,201.34,97321761.0,12531.16,12369.19,12384.66,1.64,XTR@DAX30
20190722,27171.90,17.70,226949931.0,27227.77,27088.90,27174.18,0.07,DJI@DJI,21416.79,-50.20,...,-1.37,HSI@HSI,12289.40,29.33,60318198.0,12346.02,12236.70,12243.58,0.24,XTR@DAX30
20190719,27154.20,-68.77,287691866.0,27342.96,27145.78,27246.38,-0.25,DJI@DJI,21466.99,420.75,...,1.07,HSI@HSI,12260.07,32.22,80209539.0,12340.11,12211.67,12302.64,0.26,XTR@DAX30
20190718,27222.97,3.12,218821655.0,27266.81,27068.79,27191.98,0.01,DJI@DJI,21046.24,-422.94,...,-0.46,HSI@HSI,12227.85,-113.18,75541798.0,12296.30,12172.59,12194.33,-0.92,XTR@DAX30


In [39]:
jisu_exp_info_df.shape

(4575, 72)

In [40]:
jisu_exp_info_df.isnull().sum()

종가_DJI@DJI          148
전일비_DJI@DJI         148
gvol_DJI@DJI        148
고가_DJI@DJI          148
저가_DJI@DJI          148
시가_DJI@DJI          148
rate_DJI@DJI        148
symb_DJI@DJI        148
종가_NII@NI225        263
전일비_NII@NI225       263
gvol_NII@NI225      263
고가_NII@NI225        263
저가_NII@NI225        263
시가_NII@NI225        263
rate_NII@NI225      263
symb_NII@NI225      263
종가_LNS@FTSE100      142
전일비_LNS@FTSE100     142
gvol_LNS@FTSE100    142
고가_LNS@FTSE100      142
저가_LNS@FTSE100      142
시가_LNS@FTSE100      142
rate_LNS@FTSE100    142
symb_LNS@FTSE100    142
종가_NAS@IXIC         150
전일비_NAS@IXIC        150
gvol_NAS@IXIC       150
고가_NAS@IXIC         150
저가_NAS@IXIC         150
시가_NAS@IXIC         150
                   ... 
gvol_PAS@CAC40       82
고가_PAS@CAC40         82
저가_PAS@CAC40         82
시가_PAS@CAC40         82
rate_PAS@CAC40       82
symb_PAS@CAC40       82
종가_SPI@SPX          147
전일비_SPI@SPX         147
gvol_SPI@SPX        147
고가_SPI@SPX          147
저가_SPI@SPX      

- 환율

In [41]:
base_url_exchange = 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_'

In [42]:
code_exchange = ['USDKRW', 'JPYKRW', 'EURKRW', 'CNYKRW']

In [72]:
exchange_info_df = pd.DataFrame()
for i in code_exchange :
    temp = info_collector_html(base_url_exchange, i)
    temp = temp[['날짜', '매매기준율', '전일대비']]
    temp = temp.drop_duplicates(['날짜'])
    temp.set_index('날짜', inplace=True)
    temp.columns = (temp.columns + '_' + i)
    exchange_info_df = pd.concat([exchange_info_df, temp], axis=1, sort=False)
exchange_info_df

,매매기준율_USDKRW,전일대비_USDKRW,매매기준율_JPYKRW,전일대비_JPYKRW,매매기준율_EURKRW,전일대비_EURKRW,매매기준율_CNYKRW,전일대비_CNYKRW
날짜,매매기준율,전일대비,매매기준율,전일대비,매매기준율,전일대비,매매기준율,전일대비
2019.08.01,1187.90,4.90,1087.69,1.93,1312.14,6.43,NaN,NaN
2019.07.31,1183.00,2.00,1089.62,2.09,1318.57,2.11,NaN,NaN
2019.07.30,1181.00,3.50,1087.53,2.72,1316.46,1.06,NaN,NaN
2019.07.29,1184.50,0.00,1090.25,0.20,1317.52,1.07,NaN,NaN
2019.07.26,1184.50,3.00,1090.05,3.27,1318.59,1.87,NaN,NaN
2019.07.25,1181.50,3.00,1093.32,2.57,1316.72,2.22,NaN,NaN
2019.07.24,1178.50,0.50,1090.75,0.24,1314.50,3.80,NaN,NaN
2019.07.23,1179.00,1.50,1090.51,1.03,1318.30,3.09,NaN,NaN
2019.07.22,1177.50,2.00,1091.54,0.88,1321.39,0.42,NaN,NaN


In [73]:
exchange_info_df.shape

(3807, 8)

In [74]:
exchange_info_df.isna().sum()

매매기준율_USDKRW       0
전일대비_USDKRW        0
매매기준율_JPYKRW       0
전일대비_JPYKRW        0
매매기준율_EURKRW       0
전일대비_EURKRW        0
매매기준율_CNYKRW    3803
전일대비_CNYKRW     3803
dtype: int64

- 

In [ ]:
https://finance.naver.com/
    sise/
    sise_index_day.nhn?
    code=
    KOSPI
    &page=
    {i}

In [120]:
url_creater('국내금')

['https://finance.naver.com/marketindex/goldDailyQuote.nhn?&page=']

In [86]:
aa = {'a' : 'aa', 'b':['z','x','c'], 'c':[]}
aa

{'a': 'aa', 'b': ['z', 'x', 'c'], 'c': []}

In [91]:
len(aa['b'])

3

In [79]:
type(aa['b'])

list

In [88]:
aa['c']

[]

In [ ]:
https://finance.naver.com/
    marketindex/
    worldDailyQuote.nhn?
    marketindex
    Cd=OIL_CL
    &fdtc=2&page=1

In [ ]:
https://finance.naver.com/
    marketindex/
    oilDailyQuote.nhn?
    marketindex
    Cd=OIL_GSL
    &page=1

In [ ]:
https://finance.naver.com/
    marketindex/
    worldDailyQuote.nhn?
    marketindexCd=CMDT_GC
    &fdtc=2&page=1

In [ ]:
https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page=1

In [ ]:
https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_JPYKRW&page=1

In [ ]:
https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=OIL_CL&fdtc=2&page=1

## ---------------------------------------------------------------------

In [ ]:
aa = info_collector_ajax(base_url_jisu_exp, 'DJI@DJI')
aa

In [ ]:
type(aa)

In [ ]:
target_url = base_url_jisu_exp + code_jisu_exp[0]
target_url

In [ ]:
iii = 1
page_url = (f'{target_url}&fdtc=0&page={iii}')
page_url

In [ ]:
requests.get(page_url).text

In [ ]:
requests.get(page_url).json()

In [ ]:
pd.DataFrame(requests.get(page_url).json())

In [ ]:
info_df = pd.DataFrame()

In [ ]:
info_df.append(pd.DataFrame(requests.get(page_url).json()))

In [ ]:
https://finance.naver.com/world/worldDayListJson.nhn?symbol=DJI@DJI&fdtc=0&page=2

In [ ]:
https://finance.naver.com/world/worldDayListJson.nhn?symbol=NII@NI225&fdtc=0&page=1

In [ ]:
https://finance.naver.com/world/worldDayListJson.nhn?symbol=LNS@FTSE100&fdtc=0&page=2

In [ ]:
info_collector_html(base_url_jisu_do, 'KOSPI')

#### jisu information

In [ ]:
## KOSPI, KOSDAQ, KOSPI200

In [ ]:
## 다우산업, 니케이225, 영국, 나스닥종합, 상해종합, 프랑스, S&P 500, 항셍, 독일

In [ ]:
https://finance.naver.com/sise/sise_index.nhn?code=KOSPI
https://finance.naver.com/sise/sise_index.nhn?code=KOSDAQ
https://finance.naver.com/sise/sise_index.nhn?code=KPI200

    





In [ ]:
import requests
import json

In [ ]:
uuu = 'https://finance.naver.com/world/worldDayListJson.nhn?symbol=DJI@DJI&fdtc=0&page=1'

In [ ]:
aaa = json.loads(requests.get(uuu).text)
aaa

In [ ]:
pd.DataFrame(aaa)

In [ ]:
basic_item = ['KOSPI']

In [ ]:
interest_item = ['삼성전자']

In [ ]:
input_item = basic_item + interest_item
input_item

In [ ]:
input_code = list(sto_code_sort[sto_code_sort['회사명'].isin(input_item)]['종목코드'].values)
input_code

005930

In [ ]:
sise_url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=005930'
sise_url

In [ ]:
sise_html = BeautifulSoup(urlopen(sise_url).read(), 'lxml')
sise_html

In [ ]:
page_pkg = sise_html.find_all("table",align="center")
page_pkg

In [ ]:
last_page_num = last_page_finder(sise_html)

In [ ]:
sise_exporter('KOSPI')

In [ ]:
sise_exporter('005930')

In [ ]:
sise_df = pd.DataFrame()
for i in input_code :
    sise_url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=' + i
    sise_html = BeautifulSoup(urlopen(sise_url).read(), 'lxml') #html.parser
    last_page_num = last_page_finder(sise_html)
    for j in range(1, last_page+1) :
        url = (f'{sise_url}&page={i}')
        sise_df = sise_df.append(pd.read_html(url, header=0)[0], ignore_index=True)
        
    

In [ ]:
sise_url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=' + item_code

In [ ]:
source_html = BeautifulSoup(urlopen(sise_url).read(), 'lxml') #html.parser

In [ ]:
last_page = last_page_finder(source_html)

In [ ]:
sise_df = pd.DataFrame()

In [ ]:
for i in range(1, last_page+1):
    url = (f'{sise_url}&page={i}')
    sise_df = sise_df.append(pd.read_html(url, header=0)[0], ignore_index=True)

sise_df

In [ ]:
sise_df = sise_df.dropna(axis=0)
sise_df[-5:]

In [ ]:
type(sise_df['체결가'].values[0])

In [ ]:
sise_df['날짜'] = pd.to_datetime(sise_df['날짜'])

In [ ]:
sise_df['등락률'] = sise_df['등락률'].map(lambda x: float(x.split("%")[0].strip()))

In [ ]:
sise_df.dtypes

In [ ]:
sise_df[-5:]

In [ ]:
sise_save = 'data/sise_list/' + item_code + '_' + date_key + '.csv'
sise_save

In [ ]:
sise_df.to_csv(sise_save, encoding='euc-kr', index=False) #cp949
print('save >> ' + sise_save)

In [43]:
aaa = pd.DataFrame( {
            'first_name': ['Jason', 'Jason', 'Tina', 'Jake', 'Amy'],
            'last_name': ['Miller', 'Miller', 'Ali', 'Milner', 'Cooze'],
            'age': [42, 42, 36, 24, 73],
            'preTestScore': [42, 42, 31, 2, 42],
            'postTestScore': [25, 25, 57, 62, 70]})
aaa

,first_name,last_name,age,preTestScore,postTestScore
0,Jason,Miller,42,42,25
1,Jason,Miller,42,42,25
2,Tina,Ali,36,31,57
3,Jake,Milner,24,2,62
4,Amy,Cooze,73,42,70


In [45]:
aaa[['first_name','age' ]]

,first_name,age
0,Jason,42
1,Jason,42
2,Tina,36
3,Jake,24
4,Amy,73


In [ ]:
a1 = pd.DataFrame( {
            'first_name': ['Jason', 'Bason', 'Tina', 'Jake'],
            'last_name': ['Miller', 'Miller', 'Ali', 'Milner'],
            'age': [42, 42, 36, 24] })
a1.set_index('first_name', inplace=True)
a1

In [ ]:
a2 = pd.DataFrame( {
            'first_name': ['Bason','Jake', 'Jason', 'Tina', 'Amy'],
            'last_name': ['Miller', 'Miller', 'Ali', 'Milner', 'Cooze'],
            'age': [42, 42, 36, 24, 99] })
a2.set_index('first_name', inplace=True)
a2

In [ ]:
pd.concat([a1, a2], axis=1)

In [ ]:
pd.concat([a1, a2], axis=1, sort=False)